In [1]:
!git clone -b preprocessing https://github.com/Midroni/yesweGAN

Cloning into 'yesweGAN'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 264 (delta 7), reused 70 (delta 6), pack-reused 190
Receiving objects: 100% (264/264), 388.17 MiB | 39.70 MiB/s, done.
Resolving deltas: 100% (49/49), done.
Checking out files: 100% (276/276), done.


In [2]:
!pip install tensorflow-gpu==1.15


     |████████████████████████████████| 411.5MB 35kB/s 
     |████████████████████████████████| 512kB 64.7MB/s 
  Found existing installation: tensorflow-estimator 1.15.0
    Uninstalling tensorflow-estimator-1.15.0:
      Successfully uninstalled tensorflow-estimator-1.15.0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [22]:
%cd /content
!cp -r '/content/drive/My Drive/imdb' '/content/yesweGAN/maskgan_colab/dataset'

/content


# Must be using python 2 runtime w GPU

## Run MaskGAN in MLE pretraining mode

In [25]:
%cd /content/yesweGAN/maskgan_colab
! python train_mask_gan.py \
 --data_dir='dataset/imdb' \
 --data_set='imdb' \
 --batch_size=20 \
 --sequence_length=20 \
 --base_directory='maskGAN' \
 --hparams="gen_rnn_size=650,dis_rnn_size=650,gen_num_layers=2,dis_num_layers=2,gen_learning_rate=0.00074876,dis_learning_rate=5e-4,baseline_decay=0.99,dis_train_iterations=1,gen_learning_rate_decay=0.95" \
 --mode='TRAIN' \
 --max_steps=10000 \
 --perplexity_threshold=100000 \
 --generator_model='seq2seq_vd' \
 --discriminator_model='seq2seq_vd' \
 --is_present_rate=0.5 \
 --summaries_every=10 \
 --print_every=250 \
 --max_num_to_print=3 \
 --gen_training_strategy=cross_entropy \
 --seq2seq_share_embedding=true \
 --baseline_method=critic \
 --attention_option=luong

Streaming output truncated to the last 5000 lines.
   [0]  the                   0.502   11.320 
 Sample 2.
   [0]  confused              0.504   11.365 
   [1]  attempt               0.504   0.000  
   [1]  to                    0.504   0.000  
   [0]  replicate             0.504   11.364 
   [0]  the                   0.504   11.319 
   [1]  success               0.504   0.000  
   [1]  of                    0.504   0.000  
   [0]  Turkish               0.504   11.372 
   [1]  Delight               0.504   0.000  
   [1]  does                  0.504   0.000  
   [0]  have                  0.504   11.357 
   [0]  some                  0.504   11.360 
   [1]  oddball               0.504   0.000  
   [1]  moments               0.504   0.000  
   [1]  that                  0.504   0.000  
   [1]  make                  0.504   0.000  
   [1]  it                    0.504   0.000  
   [1]  slightly              0.504   0.000  
   [1]  entertaining          0.504   0.000  
   [1]  and       

KeyboardInterrupt: ignored

In [0]:
# make copy of mle training checkpoints
#%cp /content/maskgan_colab/maskGAN/train /content/maskgan_colab/maskGAN_mle/train # make copy of mle training checkpoints

## Run MaskGAN in GAN mode

In [0]:
%cd /content/maskgan
!python train_mask_gan.py \
 --data_dir='dataset/imdb' \
 --batch_size=128 \
 --sequence_length=20 \
 --base_directory='/content/yesweGAN/maskgan_colab/maskGAN' \
 --mask_strategy=contiguous \
 --maskgan_ckpt='/content/maskgan/maskGAN/train/model.ckpt-1542'\
 --hparams="gen_rnn_size=650,dis_rnn_size=650,gen_num_layers=2,dis_num_layers=2,gen_learning_rate=0.000038877,gen_learning_rate_decay=1.0,gen_full_learning_rate_steps=2000000,gen_vd_keep_prob=0.33971,rl_discount_rate=0.89072,dis_learning_rate=5e-4,baseline_decay=0.99,dis_train_iterations=2,dis_pretrain_learning_rate=0.005,critic_learning_rate=5.1761e-7,dis_vd_keep_prob=0.71940" \
 --mode='TRAIN' \
 --max_steps=10000 \
 --generator_model='seq2seq_vd' \
 --discriminator_model='seq2seq_vd' \
 --is_present_rate=0.5 \
 --summaries_every=250 \
 --print_every=250 \
 --max_num_to_print=3 \
 --gen_training_strategy='reinforce' \
 --seq2seq_share_embedding=true \
 --baseline_method=critic \
 --attention_option=luong

/content/maskgan
<eos>: 2
<eos>: 2
Unique 2-grams: 38514
Unique 3-grams: 61239
Unique 4-grams: 69064
Vocab size: 10000
Training model.

Optimizing Generator vars:
<tf.Variable 'gen/decoder/rnn/embedding:0' shape=(10000, 650) dtype=float32_ref>
<tf.Variable 'gen/encoder/rnn/missing_embedding:0' shape=(1, 650) dtype=float32_ref>
<tf.Variable 'gen/encoder/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(1300, 2600) dtype=float32_ref>
<tf.Variable 'gen/encoder/rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(2600,) dtype=float32_ref>
<tf.Variable 'gen/encoder/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0' shape=(1300, 2600) dtype=float32_ref>
<tf.Variable 'gen/encoder/rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0' shape=(2600,) dtype=float32_ref>
<tf.Variable 'gen/decoder/attention_keys/weights:0' shape=(650, 650) dtype=float32_ref>
<tf.Variable 'gen/decoder/rnn/softmax_b:0' shape=(10000,) dtype=float32_ref>
<tf.Variable 'gen/decoder/rnn/multi_rnn_cell/cell_0/basi

## Generate samples

In [0]:
%cd /content/yesweGAN/maskgan_colab
!python generate_samples.py \
--data_dir='/content/yesweGAN/maskgan_colab/dataset/imdb' \
--data_set=ptb \
--batch_size=256 \
--sequence_length=20 \
--base_directory='/content/yesweGAN/maskgan_colab/maskGAN' \
--hparams="gen_rnn_size=650,dis_rnn_size=650,gen_num_layers=2,gen_vd_keep_prob=0.33971" \
--generator_model='seq2seq_vd' \
--discriminator_model='seq2seq_vd' \
--is_present_rate=0.0 \
--maskgan_ckpt='/content/yesweGAN/maskgan_colab/maskGAN/train/model.ckpt-2505' \
--seq2seq_share_embedding=true \
--dis_share_embedding=true \
--attention_option=luong \
--mask_strategy=contiguous \
--output_path='/content/yesweGAN/maskgan_colab' \
--baseline_method=critic \
--number_epochs=1

Streaming output truncated to the last 5000 lines.
I0224 20:16:50.410444 140570417170304 generate_samples.py:205] Evaluating on is_present_rate=0.000.
targets[[539 8 1585 2 28 113 238 54 167 10 159 1 1 3464 2 18 0 312 103 28][8 1 955 1613 6 12 1 36 12 3 21 1772 208 11 0 5412 1 1895 2595 2][4 207 2 0 3811 115 11 0 1297 13...]...]
I0224 20:16:50.476330 140570417170304 generate_samples.py:205] Evaluating on is_present_rate=0.000.
targets[[542 481 52 0 2563 8 398 1577 4 0 37 120 1 481 189 16 318 2880 8 7667][7 1 0 101 148 51 2029 1023 478 37 5031 886 555 5026 0 208 5 12 3 6][429 77 282 4625 451 248 18 12 3 6...]...]
I0224 20:16:50.539925 140570417170304 generate_samples.py:205] Evaluating on is_present_rate=0.000.
targets[[642 55 1314 11 8258 606 2 0 1162 24 5 9620 481 189 20 5635 7 8258 606 2][62 36 12 3 21 2 160 11 5031 412 5 469 2 309 4 0 655 176 1003 1895][3579 5 246 4625 451 33 185 43 3 21...]...]
I0224 20:16:50.604470 140570417170304 generate_samples.py:205] Evaluating on is_present_

## Save and Output

In [0]:
from google.colab import drive 
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# #copy checkpoints to drive
%cd /content
%cp -av maskgan /content/gdrive/My\ Drive

/content
'maskgan' -> '/content/gdrive/My Drive/maskgan'
'maskgan/checkpoint_convert.py' -> '/content/gdrive/My Drive/maskgan/checkpoint_convert.py'
'maskgan/sample_shuffler.py' -> '/content/gdrive/My Drive/maskgan/sample_shuffler.py'
'maskgan/pretrain_mask_gan.py' -> '/content/gdrive/My Drive/maskgan/pretrain_mask_gan.py'
'maskgan/README.md' -> '/content/gdrive/My Drive/maskgan/README.md'
'maskgan/dataset' -> '/content/gdrive/My Drive/maskgan/dataset'
'maskgan/dataset/iccv' -> '/content/gdrive/My Drive/maskgan/dataset/iccv'
'maskgan/dataset/iccv/ptb.valid.txt' -> '/content/gdrive/My Drive/maskgan/dataset/iccv/ptb.valid.txt'
'maskgan/dataset/iccv/ptb.train.txt' -> '/content/gdrive/My Drive/maskgan/dataset/iccv/ptb.train.txt'
'maskgan/dataset/iccv/ptb.test.txt' -> '/content/gdrive/My Drive/maskgan/dataset/iccv/ptb.test.txt'
'maskgan/dataset/iccv2017' -> '/content/gdrive/My Drive/maskgan/dataset/iccv2017'
'maskgan/dataset/iccv2017/ptb.train.txt' -> '/content/gdrive/My Drive/maskgan/datas

In [0]:
# copy from drive into runtime
%cd /content/gdrive/My\ Drive
%cp -r maskgan /content

/content/gdrive/My Drive


In [0]:
%cd /content/maskgan/maskGAN/train
from google.colab import files
files.download('model.ckpt-1542.data-00000-of-00001') 

/content/maskgan/maskGAN/train
----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 35792, 0, 0)
----------------------------------------


Traceback (most recent call last):
  File "/usr/lib/python2.7/SocketServer.py", line 293, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python2.7/SocketServer.py", line 321, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python2.7/SocketServer.py", line 334, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python2.7/SocketServer.py", line 657, in __init__
    self.finish()
  File "/usr/lib/python2.7/SocketServer.py", line 716, in finish
    self.wfile.close()
  File "/usr/lib/python2.7/socket.py", line 283, in close
    self.flush()
  File "/usr/lib/python2.7/socket.py", line 307, in flush
    self._sock.sendall(view[write_offset:write_offset+buffer_size])
error: [Errno 32] Broken pipe


In [0]:
def _read_words(filename, use_prefix=True):
  all_words = []
  sequence_example = tf.train.SequenceExample()
  for r in tf.python_io.tf_record_iterator(filename):
    sequence_example.ParseFromString(r)

    if FLAGS.prefix_label and use_prefix:
      label = sequence_example.context.feature['class'].int64_list.value[0]
      review_words = [EOS_INDEX + 1 + label]
    else:
      review_words = []
    review_words.extend([
        f.int64_list.value[0]
        for f in sequence_example.feature_lists.feature_list['token_id'].feature
    ])
    all_words.append(review_words)
  return all_words

In [0]:
def build_vocab(vocab_file):
  word_to_id = {}

  with tf.gfile.GFile(vocab_file, 'r') as f:
    index = 0
    for word in f:
      word_to_id[word.strip()] = index
      index += 1
    word_to_id['<eos>'] = EOS_INDEX

  return word_to_id


def imdb_raw_data(data_path=None):
  """Load IMDB raw data from data directory "data_path".
  Reads IMDB tf record files containing integer ids,
  and performs mini-batching of the inputs.
  Args:
    data_path: string path to the directory where simple-examples.tgz has
      been extracted.
  Returns:
    tuple (train_data, valid_data)
    where each of the data objects can be passed to IMDBIterator.
  """

  train_path = os.path.join(data_path, 'train_lm.tfrecords')
  valid_path = os.path.join(data_path, 'test_lm.tfrecords')

  train_data = _read_words(train_path)
  valid_data = _read_words(valid_path)
  return train_data, valid_data

In [20]:
import os
imdb_raw_data('dataset/imdb')

NameError: ignored